<a href="https://colab.research.google.com/github/5dongdong/PY_AI/blob/master/model%20/LightGBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import MinMaxScaler, StandardScaler, MaxAbsScaler, RobustScaler
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings('ignore')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df = pd.read_csv('/content/drive/MyDrive/data/medical_noshow.csv')  

In [4]:
# AppointmentDay - ScheduledDay 값을 계산하여 'DateDifference' 새로운 열 생성
df['DateDifference'] = (pd.to_datetime(df['AppointmentDay']).dt.date - pd.to_datetime(df['ScheduledDay']).dt.date).dt.days

df.drop(['ScheduledDay', 'AppointmentDay', 'PatientId', 'AppointmentID'], axis=1, inplace=True)

# object 열 추출
object_columns = df.select_dtypes(include='object').columns

# LabelEncoder 객체 생성
encoder = LabelEncoder()

# LabelEncoder를 사용하여 object 열 변환
for column in object_columns:
    df[column] = encoder.fit_transform(df[column])

In [5]:
df

,Gender,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show,DateDifference
0,0,62,39,0,1,0,0,0,0,0,0
1,1,56,39,0,0,0,0,0,0,0,0
2,0,62,45,0,0,0,0,0,0,0,0
3,0,8,54,0,0,0,0,0,0,0,0
4,0,56,39,0,1,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
110522,0,56,43,0,0,0,0,0,1,0,35
110523,0,51,43,0,0,0,0,0,1,0,35
110524,0,21,43,0,0,0,0,0,1,0,41
110525,0,38,43,0,0,0,0,0,1,0,41


In [6]:
x = df[['Gender', 'Age', 'Neighbourhood', 'Scholarship', 'Hipertension','Diabetes', 'Alcoholism', 'Handcap', 'SMS_received', 'DateDifference']]
y = df[['No-show']]   

x_train, x_test, y_train,y_test = train_test_split(
    x, y, train_size=0.8, shuffle=True, random_state=42
)
     

In [7]:
# 변수 설정
n_splits = 11    # 보통 홀수로 들어감
random_state = 42
scaler = MinMaxScaler()

In [8]:
kfold = KFold(n_splits=n_splits, shuffle=True, 
              random_state=random_state)


# Scaler
scaler.fit(x_train)                 
x = scaler.transform(x_train)   # train 은 fit, transform 모두 해줘야 함
x = scaler.transform(x_test) 

In [9]:
# 2. 모델
model = LGBMClassifier()

# 3. 훈련
model.fit(x_train, y_train)

#4. 평가, 예측
result = model.score(x_test,y_test)

score = cross_val_score( model, 
                        x_train, y_train,
                        cv = kfold )  #cv = cross validation_

y_predict = cross_val_predict(model,
                              x_test, y_test,
                              cv = kfold)

acc = accuracy_score(y_test, y_predict)

print('결과 acc : ', result)
print('cv pred acc : ', acc )

# 결과 acc :  0.8002804668415815
# cv pred acc :  0.7979281642992853



결과 acc :  0.8002804668415815
cv pred acc :  0.7979281642992853
